In [1]:
from pathlib import Path
import json
import sys

from neuvueclient import NeuvueQueue
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

sys.path.append(str(Path.cwd().parents[1]))
from utils.constants import NEUVUE_QUEUE_URL

In [2]:
client = NeuvueQueue(NEUVUE_QUEUE_URL)

Auth method: Config File


## Generic Forced Choice Task

Task type for a forced choice task.

In [3]:
# Change these as needed
AUTHOR = "dxenes1"
NAMESPACE = "forcedChoiceExample"
ASSIGNEES = ["unassigned_novice"]
INSTRUCTIONS = {"prompt": "Is the selected seg ID a multi-soma? (two valid neurons merged together)"}


### Grab the necessary input data, in this case we are just using the nucleus detection table.

In [4]:
from caveclient import CAVEclient
cave = CAVEclient('minnie65_phase3_v1')

In [5]:
nucleus_neuron_svm = cave.materialize.query_table('nucleus_neuron_svm')
nucleus_neuron_svm['pt_position_8nm'] = nucleus_neuron_svm['pt_position'].apply(lambda x: np.array(x) // np.array([2,2,1])) 

Table Owner Notice on nucleus_neuron_svm: Please cite https://doi.org/10.1101/2022.07.20.499976 when using this table.


In [6]:
# Filter only neurons and get multi-soma neurons by using value counts on the root IDs
neuron_nuclei = nucleus_neuron_svm[nucleus_neuron_svm['cell_type'] == 'neuron']
multi_soma_neurons = neuron_nuclei['pt_root_id'].value_counts()[neuron_nuclei['pt_root_id'].value_counts() >= 2]

# Remove seg ID 0 and 864691135763133238 (the MOAME - mother of all merge errors)
multi_soma_neurons.drop([0, 864691135763133238], inplace=True)



### Create task dataframe to more easily create and assign tasks.

In [7]:
task_rows = []
for root_id in multi_soma_neurons.index.unique():
    nuclei_subset = neuron_nuclei[neuron_nuclei['pt_root_id']==root_id]
    row = {}
    row['pt_root_id'] = root_id
    row['proofreaders'] = ASSIGNEES
    row['metadata'] = {'nuclei': nuclei_subset['id'].tolist()}
    row['pt_position'] = nuclei_subset.iloc[0].pt_position_8nm
    row['annos'] = nuclei_subset['pt_position_8nm'].tolist()
    row['instructions'] = INSTRUCTIONS
    task_rows.append(row)
    
task_df = pd.DataFrame(task_rows)

In [8]:
task_df.head()

,pt_root_id,proofreaders,metadata,pt_position,annos,instructions
0,864691135968943973,[unassigned_novice],"{'nuclei': [531788, 525961, 528862, 528334, 52...","[150720, 105976, 15961]","[[150720, 105976, 15961], [152784, 96480, 1492...",{'prompt': 'Is the selected seg ID a multi-som...
1,864691135270095013,[unassigned_novice],"{'nuclei': [198054, 198124, 198037, 198128, 19...","[68304, 91832, 27724]","[[68304, 91832, 27724], [70448, 94480, 27295],...",{'prompt': 'Is the selected seg ID a multi-som...
2,864691135479319750,[unassigned_novice],"{'nuclei': [208610, 276973, 208491, 176123, 37...","[71696, 131048, 26468]","[[71696, 131048, 26468], [82080, 132248, 26741...",{'prompt': 'Is the selected seg ID a multi-som...
3,864691135888983689,[unassigned_novice],"{'nuclei': [608855, 608854, 611313, 642226]}","[177280, 51616, 25484]","[[177280, 51616, 25484], [177000, 54856, 25556...",{'prompt': 'Is the selected seg ID a multi-som...
4,864691136974041116,[unassigned_novice],"{'nuclei': [592840, 495316, 594720, 497173]}","[164848, 108232, 21331]","[[164848, 108232, 21331], [146232, 96216, 2160...",{'prompt': 'Is the selected seg ID a multi-som...


### We use a function in ng_utils.py to parse this dataframe and create custom neuroglancer states for each task.

In [9]:
from utils.ng_state_utils import add_basic_ng_states_with_anno_layer_and_points_to_df
add_basic_ng_states_with_anno_layer_and_points_to_df(task_df)

/Users/xenesd1/Projects/MICrONS/neuvue-manage/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### We finally upload the tasks using another convenience function which reads in the data frame and creates the tasks sequentially.

The NeuVue client will automatically upload JSON state strings to the JSON state server as its creating the tasks. 

In [10]:
from utils.post_tasks_utils import post_tasks_from_state_and_proofreader_df
post_tasks_from_state_and_proofreader_df(
    neuvue_client=client, 
    state_df=task_df, 
    author=AUTHOR, 
    namespace=NAMESPACE
)
print("Done")

41it [00:12,  3.30it/s]

Done


## Delete all tasks
If something went wrong you can use this snippet to delete all tasks from this namespace

In [11]:
# tasks = client.get_tasks(sieve={'namespace': NAMESPACE, 'assignee': ASSIGNEES}, select=['_id'])
# for _id in tasks.index:
#     client.delete_task(_id)